In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression,Ridge, Lasso, RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score
import csv

In [3]:
RANDOM_STATE = 42

In [4]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
%%time
lr = LinearRegression()
lr.fit(X_test, y_test)
print (f'R2_lr: {lr.score(X_test, y_test):.4f}')

ridge = Ridge(random_state=42)
ridge.fit(X_test, y_test)
print (f'R2_ridge: {ridge.score(X_test, y_test):.4f}')

lasso = Lasso(random_state=42)
lasso.fit(X_test, y_test)
print (f'R2_lasso: {lasso.score(X_test, y_test):.4f}')

R2_lr: 0.8734
R2_ridge: 0.8707
R2_lasso: 0.8140
Wall time: 84.6 ms


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
parametres = {'alpha': [10**(-5), 10**(-4), 10**(-3), 10**(-2), 10**(-1), 10**(1), 10**(2), 10**(3), 10**(4), 10**(5)]}

ridgeCV = GridSearchCV(ridge, parametres, scoring='r2')
lassoCV = GridSearchCV(lasso, parametres, scoring='r2')

ridgeCV_fit = ridgeCV.fit(X_test, y_test)
lassoCV_fit = lassoCV.fit(X_test, y_test)

print(f'R2_ridgeCV_fit: {ridgeCV_fit.score(X_test, y_test)},' f'R2_lassoCV_fit: {lassoCV_fit.score(X_test, y_test)}')


#исходя из полученных параметров можно сделать вывод, что вновь подобранные параметры являются более качественными по сравнении с предыдущим результатом

R2_ridgeCV_fit: 0.8603467111466161,R2_lassoCV_fit: 0.8683665834820338


4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [8]:
ridge_pipeline_standart = Pipeline([('StandardScaler', StandardScaler()), ('svc', Ridge(random_state=42))])
ridge_pipeline_minmax = Pipeline([('MinMaxScaler', MinMaxScaler()), ('svc', Ridge(random_state=42))])
lasso_pipeline_standart = Pipeline([('StandardScaler', StandardScaler()), ('svc', Lasso(random_state=42))])
lasso_pipeline_minmax = Pipeline([('MinMaxScaler', MinMaxScaler()), ('svc', Lasso(random_state=42))])

ridge_pipeline_standart_fit = ridge_pipeline_standart['svc'].fit(X_test, y_test)
ridge_pipeline_minmax_fit = ridge_pipeline_minmax['svc'].fit(X_test, y_test)
lasso_pipeline_standart_fit = lasso_pipeline_standart['svc'].fit(X_test, y_test)
lasso_pipeline_minmax_fit = lasso_pipeline_minmax['svc'].fit(X_test, y_test)


print(f'R2_ridge_pipeline_standart_fit: {ridge_pipeline_standart_fit.score(X_test, y_test)}')
print(f'R2_ridge_pipeline_minmax_fit: {ridge_pipeline_minmax_fit.score(X_test, y_test)}')
print(f'R2_lasso_pipeline_standart_fit: {lasso_pipeline_standart_fit.score(X_test, y_test)}')
print(f'R2_lasso_pipeline_minmax_fit: {lasso_pipeline_minmax_fit.score(X_test, y_test)}')

#по сравнению с результатами до масштабирования, результаты для ridge увеличились, а для lasso уменьшились

R2_ridge_pipeline_standart_fit: 0.8707310383105309
R2_ridge_pipeline_minmax_fit: 0.8707310383105309
R2_lasso_pipeline_standart_fit: 0.814041753594029
R2_lasso_pipeline_minmax_fit: 0.814041753594029


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [9]:
parametres = {'alpha': [10**(-5), 10**(-4), 10**(-3), 10**(-2), 10**(-1), 10**(1), 10**(2), 10**(3), 10**(4), 10**(5)]}

classifiers = [
    ('Ridge', Ridge(random_state=42), parametres),
    ('Lasso', Lasso(random_state=42), parametres)
]

ridge_pipeline_standart = Pipeline([('StandardScaler', StandardScaler()), ('svc', Ridge(random_state=42))])
ridge_pipeline_minmax = Pipeline([('MinMaxScaler', MinMaxScaler()), ('svc', Ridge(random_state=42))])
lasso_pipeline_standart = Pipeline([('StandardScaler', StandardScaler()), ('svc', Lasso(random_state=42))])
lasso_pipeline_minmax = Pipeline([('MinMaxScaler', MinMaxScaler()), ('svc', Lasso(random_state=42))])

ridgeCV = GridSearchCV(ridge, parametres, scoring='r2')
lassoCV = GridSearchCV(lasso, parametres, scoring='r2')

ridgeCV_fit = ridgeCV.fit(X_test, y_test)
lassoCV_fit = lassoCV.fit(X_test, y_test)

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [10]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [11]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [12]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [13]:
x = data.loc[:,:13]
y = data[14].map({'<=50K': 0, '>50K': 1})


9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [14]:
x.info()
#Пропуски отсутствуют, т.к. все поля имеют значение non-null, следовательно, заменять нечего

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       48842 non-null  int64 
 1   1       48842 non-null  object
 2   2       48842 non-null  int64 
 3   3       48842 non-null  object
 4   4       48842 non-null  int64 
 5   5       48842 non-null  object
 6   6       48842 non-null  object
 7   7       48842 non-null  object
 8   8       48842 non-null  object
 9   9       48842 non-null  object
 10  10      48842 non-null  int64 
 11  11      48842 non-null  int64 
 12  12      48842 non-null  int64 
 13  13      48842 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


10. Выберите колонки с числовыми и категориальными переменными.

In [15]:
nums = x.select_dtypes(include=['int64']).columns.tolist()
cats = x.select_dtypes(include=['object']).columns.tolist()
print(nums, cats)

[0, 2, 4, 10, 11, 12] [1, 3, 5, 6, 7, 8, 9, 13]


11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [16]:
col_transform = ColumnTransformer([
    ('scale',  MinMaxScaler(), nums),
    ('onehot', OneHotEncoder(),  cats)])

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [17]:
y.value_counts()

0    37155
1    11687
Name: 14, dtype: int64

In [22]:
accuracy = accuracy_score (y, [0 for i in range(len(y))])
f1 = f1_score (y, [0 for i in range(len(y))])
print(accuracy, f1)


0.7607182343065395 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [23]:
question_mark = SimpleImputer(missing_values='?', strategy='most_frequent')

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.